<a href="https://colab.research.google.com/github/giuliovv/ANNDL_competition_1/blob/master/giulio_models/nasnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

colab = "True" #@param ['True','False']
if colab == "True":
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/Colab Notebooks/
  if not os.path.isdir('training'):
    !unzip dataset.zip


Mounted at /gdrive
/gdrive/MyDrive/Colab Notebooks


In [2]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
dataset_dir = 'training'
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import shutil
if not "test" in os.listdir():
  print("No test directory!")
  for label in labels:
    print(label)
    image_names = [pic for pic in os.listdir("training/"+label)]
    train_images, test_images = train_test_split(image_names, test_size=0.05)
    if not 'test/'+label in os.listdir():
      try:
        os.makedirs(os.path.abspath(os.getcwd())+'/test/'+label+'/')
      except OSError:
        print("Failed to create a dir for ", '/test/'+label+'/')
      else:
        print("Success creating folder ", label)
    for name in test_images:
      shutil.move(os.path.abspath(os.getcwd())+'/training/'+label+'/'+name, os.path.abspath(os.getcwd())+'/test/'+label+'/'+name)
  print("Transfered all testing data!")

In [6]:
import tensorflow as tf
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE = 64
VALIDATE_BATCH_SIZE = BATCH_SIZE

train_generator = ImageDataGenerator(
## when rotating some pixels remain unfilled, to fill them the default function is to take the "nearest pixel color", this maybe screws up the learning process so I left it for later training
##                                     rotation_range=90,
                                    width_shift_range=0.1, 
                                    height_shift_range=0.1,
                                    horizontal_flip=True, 
                                    vertical_flip=True,
                                    validation_split=0.2,
                                    preprocessing_function=tf.keras.applications.vgg16.preprocess_input) # VGG16 preprocessing
test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

traingen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='training',
                                              batch_size=BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

validgen = train_generator.flow_from_directory('training',
                                              target_size=(256, 256),
                                              class_mode='categorical',
                                              classes=labels,
                                              subset='validation',
                                              batch_size=VALIDATE_BATCH_SIZE,
                                              shuffle=True,
                                              seed=42)

testgen = test_generator.flow_from_directory('test',
                                            target_size=(256, 256),
                                            class_mode='categorical',
                                            classes=labels,
                                            batch_size=1,
                                            shuffle=False,
                                            seed=42)

In [ ]:
base_model = NASNetMobile(weights='imagenet', include_top=False)

In [ ]:
n_classes = len(labels)

In [ ]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have n_classes classes
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze original layers
model.trainable = True    
set_trainable = False
for layer in model.layers:
  if layer.name == 'activation_166':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
  print("layer {} is {}".format(layer.name, '+++trainable' if layer.trainable else '---frozen'))


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.experimental.BackupAndRestore(
    "nasnet"
)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "./logs"

In [ ]:
n_steps = traingen.samples / BATCH_SIZE
n_val_steps = validgen.samples / VALIDATE_BATCH_SIZE
n_epochs = 2

model.fit(traingen,
      epochs=n_epochs,
      validation_data=validgen,
      steps_per_epoch=n_steps,
      validation_steps=n_val_steps,
      verbose=1,
      callbacks=[callback, tensorboard_callback])

In [ ]:
model.save("nasnet")